<a href="https://colab.research.google.com/github/clyde2020/ML_Portfolio/blob/main/HCD_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import libraries

In [1]:
import pandas as pd
import numpy as np
import os
import shutil
from shutil import copyfile
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop, Adam
import matplotlib.pyplot as plt
import random
!pip install keras-tuner -q
from keras_tuner import RandomSearch
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import load_model
import pickle
from keras.models import load_model
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import auc, roc_curve
from keras.callbacks import ReduceLROnPlateau

from google.colab import drive
drive.mount('/content/gdrive')

     |████████████████████████████████| 98 kB 3.3 MB/s 
Mounted at /content/gdrive


## Initialize variables

In [2]:
df = pd.read_csv('/content/gdrive/MyDrive/Histopath/train_labels.csv')

def append_ext(file):
  return file + '.tif'

def remove_ext(file):
  return file.split('.')[0]

# Define the folder to pull images from - 
# Data split is faster with smaller source folder
source_folder = '/content/gdrive/MyDrive/Histopath/120k_set'

# It takes awhile for this list to load, so I pickled it for convenience
with open('/content/gdrive/MyDrive/Histopath/Saved_data/120k_tif_source_list', 'rb') as f:
  tif_source_list = pickle.load(f)

state = 60
home_dir = '/content/gdrive/MyDrive/Histopath/Large_96K_total1'

We can use the below commented out code if we don't use pickled data

In [ ]:
# tif_source_list = os.listdir(source_folder)

In [ ]:
# with open('/content/gdrive/MyDrive/Histopath/Saved_data/120k_tif_source_list', 'wb') as f:
#   pickle.dump(tif_source_list, f)

## Process data

In [3]:
source_list = map(remove_ext, tif_source_list)
df_sample = df[df['id'].isin(source_list)]
picdict = dict(zip(df_sample['id'], df_sample['label']))

How many of each class are represented?

In [4]:
df_sample['label'].value_counts()

0    71573
1    48427
Name: label, dtype: int64

In [ ]:
sample_size = 48000  # Update for train

# Sample a number of each label type
df_0_train = df_sample[df_sample['label'] == 0].sample(sample_size)
df_1_train = df_sample[df_sample['label'] == 1].sample(sample_size)

# Put into single dataframe
df_train_full = pd.concat([df_0_train, df_1_train], axis=0).reset_index(drop=True)

# Add the extension
df_train_full['id'] = df_train_full['id'].apply(append_ext)

# Shuffle the dataframe
df_train_full = shuffle(df_train_full)

In [ ]:
# Split into train, valid split lists
df_train, df_valid = train_test_split(df_train_full, 
                                      test_size=10000, 
                                      random_state=state, 
                                      stratify=df_train_full['label'])
train_pre_list = list(df_train['id'])
valid_pre_list = list(df_valid['id'])

# Get id lists
df_source = pd.DataFrame(tif_source_list, columns=['id'])
df_train_list = df_source[df_source.id.isin(train_pre_list)]
df_valid_list = df_source[df_source.id.isin(valid_pre_list)]

train_list = list(df_train_list['id'])
valid_list = list(df_valid_list['id'])

# Make sure lists have correct # of elements
print(len(train_list), len(valid_list))
assert (len(train_list) + len(valid_list)) == (2 * sample_size)

86000 10000


Set data folder variables

In [ ]:
# Update these variables according to the dataset to use
train_comp = '{}/Train'.format(home_dir)
valid_comp = '{}/Valid'.format(home_dir)
benign_train_comp = '{}/Benign'.format(train_comp)
benign_valid_comp = '{}/Benign'.format(valid_comp)
mal_train_comp = '{}/Malignant'.format(train_comp)
mal_valid_comp = '{}/Malignant'.format(valid_comp)

Folder creation before split data

In [ ]:
os.mkdir(home_dir)
os.mkdir(train_comp)
os.mkdir(valid_comp)
os.mkdir(benign_train_comp)
os.mkdir(benign_valid_comp)
os.mkdir(mal_train_comp)
os.mkdir(mal_valid_comp)

This function splits the dataset into malignant and benign

In [ ]:
def split_data(LIST, SOURCE, MAL_FOLDER, BENIGN_FOLDER):
  for fname in LIST:
    origin = os.path.join(SOURCE, fname)
    name = remove_ext(fname)
    if picdict[name] == 0:
      shutil.copy(src=origin, dst=BENIGN_FOLDER)
    else:
      shutil.copy(src=origin, dst=MAL_FOLDER)

Execute the data split

In [ ]:
split_data(valid_list, source_folder, mal_valid_comp, benign_valid_comp)
split_data(train_list, source_folder, mal_train_comp, benign_train_comp)

Data Augmentation

In [ ]:
batch_size = 32

train_datagen = ImageDataGenerator(rescale=1/255.0,
                                  rotation_range=45,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  shear_range=0.2,
                                  horizontal_flip=True,
                                  fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(
    train_comp,
    target_size=(96, 96),
    batch_size=batch_size,
    class_mode='binary')

validation_datagen = ImageDataGenerator(rescale=1/255.0)  

validation_generator = validation_datagen.flow_from_directory(
    valid_comp,
    target_size=(96, 96),
    batch_size=batch_size,
    class_mode='binary') 

Found 86400 images belonging to 2 classes.
Found 9600 images belonging to 2 classes.


## Define and execute the model

In [ ]:
state = 75

tf.keras.backend.clear_session()
tf.random.set_seed(state)

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(filters=32, 
                            kernel_size=3, 
                            input_shape=(96, 96, 3), 
                            activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.Conv2D(filters=32, 
                            kernel_size=3, 
                            activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.MaxPooling2D(2),
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Conv2D(filters=64, 
                            kernel_size=3, 
                            activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.Conv2D(filters=64, 
                            kernel_size=3, 
                            activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.MaxPooling2D(2),
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Conv2D(filters=128, 
                            kernel_size=3, 
                            activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.Conv2D(filters=128, 
                            kernel_size=3, 
                            activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.MaxPooling2D(2),
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=256, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

optimizer = tf.keras.optimizers.Adam(learning_rate=5e-3)

model.compile(optimizer=optimizer, 
              loss='binary_crossentropy', 
              metrics=['acc']
              )

reduce_lr = ReduceLROnPlateau(monitor='acc',
                              factor=0.5,
                              patience=1,
                              cooldown=2,
                              min_lr=1e-4
                              )

mc = ModelCheckpoint(monitor='val_acc', 
                     filepath='/content/gdrive/MyDrive/Histopath/Saved_models/Large_96k_Reduce_LR_6Layer_bestmodel.h5', 
                     verbose=1, 
                     save_best_only=True, 
                     mode='max'
                     )

cd = [mc, reduce_lr]

In [ ]:
history = model.fit(train_generator,
                    epochs=24,
                    verbose=1,
                    validation_data=validation_generator,
                    callbacks=cd
                    )

## Get Predictions and create Submission CSV file

In [ ]:
model = load_model('/content/gdrive/MyDrive/Histopath/Saved_models/Large_96k_Reduce_LR_6Layer_bestmodel.h5')
y_pred = pd.read_csv('/content/gdrive/MyDrive/Histopath/blank_sample_submission.csv')
test_comp = '/content/gdrive/MyDrive/Histopath/test'
y_pred['id'] = y_pred['id'].apply(append_ext)

In [ ]:
# test_files = pd.DataFrame({'file_name': os.listdir(test_comp)})
# with open('/content/gdrive/MyDrive/Histopath/full_test_files', 'wb') as f:
#   pickle.dump(test_files, f)

with open('/content/gdrive/MyDrive/Histopath/Saved_data/full_test_files', 'rb') as f:
  test_files = pickle.load(f)

In [ ]:
test_datagen = ImageDataGenerator(rescale=1/255.0)  

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_files,
    directory=test_comp,
    x_col='file_name',
    target_size=(96, 96),
    batch_size=1,
    class_mode=None,
    shuffle=False) 

Found 57458 validated image filenames.


In [ ]:
len_test = len(test_files)
len_test

57458

In [ ]:
preds = model.predict(test_generator,
                      steps=len_test,
                      verbose=1)

57458/57458 [==============================] - 9576s 167ms/step


Process the prediction labels

In [ ]:
preds = preds[:, 0]
preds = np.round(preds)
preds.astype(int)

array([0, 1, 1, ..., 0, 0, 1])

Merge the prediction labels into the submission dataframe

In [ ]:
test_file_names = test_generator.filenames
df_preds = pd.DataFrame({'id': test_file_names, 'label': preds})
y_pred = y_pred.merge(df_preds, on='id')

Get submission dataframe into correct format

In [ ]:
y_pred['label'] = y_pred['label_y']
y_pred.drop(['label_x'], axis=1, inplace=True)
y_pred.drop(['label_y'], axis=1, inplace=True)
y_pred['id'] = y_pred['id'].apply(remove_ext)

In [ ]:
y_pred.to_csv('/content/gdrive/MyDrive/Histopath/sample_submission.csv', index=False)

## Get AUC

In [ ]:
val_test_generator = validation_datagen.flow_from_directory(
    valid_comp,
    target_size=(96, 96),
    batch_size=1,
    class_mode='binary',
    shuffle=False) 

Found 3000 images belonging to 2 classes.


In [ ]:
len_val_test = len(val_test_generator)

val_test_pred = model.predict(val_test_generator, 
                         steps=len_val_test,
                         verbose=1)[:, 0]

3000/3000 [==============================] - 20s 7ms/step


In [ ]:
val_test_true = val_test_generator.classes
val_test_pred = np.round(val_test_pred)

In [ ]:
val_test_pred.astype(int)

array([0, 0, 0, ..., 1, 1, 1])

In [ ]:
acc_val_test = np.equal(val_test_pred, val_test_true).sum() / len(val_test_pred)
print('Validation accuracy: {:.3f}'.format(acc_val_test))

In [ ]:
fpr, tpr, thresholds = roc_curve(val_test_true, val_test_pred)

In [ ]:
auc_val_test = auc(fpr, tpr)
print('Validation AUC: {:.3f}'.format(auc_val_test))